<a href="https://colab.research.google.com/github/Urzelhen/NLP_streamlit_GPT_QA/blob/main/pushkin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Неделя 7: NLP

## Генерация текста с помощью модели **GPT**

https://www.kaggle.com/tuckerarrants/text-generation-with-huggingface-gpt2

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q transformers

     |████████████████████████████████| 4.9 MB 4.8 MB/s 
     |████████████████████████████████| 6.6 MB 46.5 MB/s 
     |████████████████████████████████| 120 kB 75.6 MB/s 


In [3]:
import numpy as np
import pandas as pd
import re
import random

import torch
from tqdm.notebook import tqdm
import transformers
from torch.optim import AdamW

device = 'cuda' if torch.cuda.is_available() else 'cpu'

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [4]:
# Загружаем токенайзер модели
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')


Downloading:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

In [5]:
# Загружаем файл с текстом, на котором будем дообучать модель
import re
# with open('/content/drive/MyDrive/avidreaders.ru__evgeniy-onegin.txt', encoding='utf8') as f:
#     text_onegin = f.read()


# with open('/content/drive/MyDrive/Zov-Ktulhu txt.txt', encoding='utf8') as f:
#     text_ktulhu = f.read()

# with open('/content/drive/MyDrive/Izgonyaushiy', encoding='utf8') as f:
#     text_izgonyaushiy = f.read()

# with open('/content/drive/MyDrive/tired_blood', encoding='utf8') as f:
#     text_tired_blood = f.read()

# with open('/content/drive/MyDrive/morgen', encoding='utf8') as f:
#     text_morgen = f.read()

# with open('/content/drive/MyDrive/50_shades_of_grey', encoding='utf8') as f:
#     text_50_shades = f.read()

# with open('/content/drive/MyDrive/horoscope', encoding='utf8') as f:
#     text_horoscope = f.read()

# text = '' + text_morgen + text_horoscope
#  + text_tired_blood

with open('/content/drive/MyDrive/pushkin', encoding='utf8') as f:
    text = f.read()

# New Section

In [6]:
text.find('[')

-1

In [7]:
print(text[1000:])


"У вас товар, у нас купец:
Собою парень молодец,
И статный, и проворный,
Не вздорный, не зазорный.
Богат, умен, ни перед кем
Не кланяется в пояс,
А как боярин между тем
Живет, не беспокоясь;
А подарит невесте вдруг
И лисью шубу, и жемчуг,
И перстни золотые,
И платья парчевые.
Катаясь, видел он вчера
Ее за воротами;
Не по рукам ли, да с двора,
Да в церковь с образами?"
Она сидит за пирогом
Да речь ведет обиняком,
А бедная невеста
Себе не видит места.
"Согласен,-- говорит отец,--
Ступай благополучно,
Моя Наташа, под венец:
Одной в светелке скучно.
Не век девицей вековать,
Не все касатке распевать,
Пора гнездо устроить,
Чтоб детушек покоить".
Наташа к стенке уперлась
И слово молвить хочет --
Вдруг зарыдала, затряслась,
И плачет, и хохочет.
В смятенье сваха к ней бежит,
Водой студеною поит
И льет остаток чаши
На голову Наташи.
Крушится, охает семья.
Опомнилась Наташа
И говорит: "Послушна я,
Святая воля ваша.
Зовите жениха на пир.
Пеките хлебы на весь мир,
На славу мед варите
Да суд на пир

In [8]:
roman_numbers = 'IXLVM'
chapters = ["Глава первая", "Глава вторая", "Глава третья", "Глава четвертая", "Глава пятая", "Глава шестая", "Глава седьмая", "Глава восьмая", "I. Ужас в глине", "II. Рассказ инспектора Легресса", "III. Морское безумие"]
for chapter in chapters:
  text.replace(chapter, ' ')
text = text.translate(str.maketrans('', '', roman_numbers))
# text = text.translate(str.maketrans('', '', chapters))
text = text.replace("\n* * *\n", '\n')
text = text.replace("--", "—")
text = text.replace('" ', '"')
text = re.sub('\n{2,}', '\n', text)
text = re.sub('[\d]', '', text)
text = text.replace("[]", '')

In [9]:
print(text[1000:])

 вас товар, у нас купец:
Собою парень молодец,
И статный, и проворный,
Не вздорный, не зазорный.
Богат, умен, ни перед кем
Не кланяется в пояс,
А как боярин между тем
Живет, не беспокоясь;
А подарит невесте вдруг
И лисью шубу, и жемчуг,
И перстни золотые,
И платья парчевые.
Катаясь, видел он вчера
Ее за воротами;
Не по рукам ли, да с двора,
Да в церковь с образами?"
Она сидит за пирогом
Да речь ведет обиняком,
А бедная невеста
Себе не видит места.
"Согласен,— говорит отец,—
Ступай благополучно,
Моя Наташа, под венец:
Одной в светелке скучно.
Не век девицей вековать,
Не все касатке распевать,
Пора гнездо устроить,
Чтоб детушек покоить".
Наташа к стенке уперлась
И слово молвить хочет —
Вдруг зарыдала, затряслась,
И плачет, и хохочет.
В смятенье сваха к ней бежит,
Водой студеною поит
И льет остаток чаши
На голову Наташи.
Крушится, охает семья.
Опомнилась Наташа
И говорит: "Послушна я,
Святая воля ваша.
Зовите жениха на пир.
Пеките хлебы на весь мир,
На славу мед варите
Да суд на пир зовит

In [10]:
# токенизируем текст
tokens = tokenizer.encode(text, add_special_tokens=True)
tokens = np.array(tokens)
print(len(tokens))
tokens[:10]

45494


array([20377,  2299, 20361,  3384,  6304,   203,  2176,   354,   669,
       30618])

In [11]:
# разбиваем на train и test

l = len(tokens)//15
train = []
test = []
for i in range(15):
    if i%5 > 0:
        train.extend(tokens[i*l: (i+1)*l])
    else:
        test.extend(tokens[i*l: (i+1)*l])
train = np.array(train)
test = np.array(test)

print(len(tokens), len(train), len(test))

45494 36384 9096


In [12]:
# re.split('; |, ', string_to_split)

In [13]:
from transformers import GPT2LMHeadModel

# Эту модель просто подгружаем и не будем дообучать 
model_init = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)


# Эту модель подгрузим и далее обучим
model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device);
model_init.to(device);

Downloading:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [14]:
batch_size = 16
max_len = 256
epochs = 10

n_train = len(train)//(batch_size*max_len)
n_test = len(test)//(batch_size*max_len)
print(n_train, n_test)

# устанавливаем оптимизатор
optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

# трансформеры с трудом обучаются, для них нужны разные способы повышения 
# эффективности градиентного спуска
total_steps = n_train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)


def accuracy(y_true, logits):
    return torch.mean((y_true[1:] == torch.argmax(logits, dim=2)[:-1]).float()).detach().cpu().numpy()

8 2


In [15]:
# готовим тензоры для обучения, поскольку не пользуемся DataLoader

def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids


# обучающий цикл
for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    train_loss = []
    train_acc = []
    model.train()
    pbar = tqdm(range(n_train))
    for i in pbar:
        batch_ids = prep_tensors(train, i)

        model.zero_grad() # = optimizer.zero_grad()
        loss, logits, _ = model(batch_ids, # logits - что подаем на SoftMax
                                token_type_ids=None, 
                                
                                labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # нормируем веса, что норма была 1
        optimizer.step()
        scheduler.step()
        
        train_loss.append(loss.item())
        train_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(train_acc):.4f} loss {np.mean(train_loss):.4f}', refresh=True)

    
    print(f'epoch {epoch}/{epochs} : validation')
    model.eval()
    val_acc = []
    val_loss = []
    pbar = tqdm(range(n_test))
    for i in pbar:
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():        
            loss, logits, _ = model(batch_ids, 
                                token_type_ids=None, 
                                # attention_mask=batch_mask,
                                labels=batch_ids
                                 ).values()
        
        val_loss.append(loss.item())
        val_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(val_acc):.4f} loss {np.mean(val_loss):.4f}', refresh=True)


epoch 1/10 : training


  0%|          | 0/8 [00:00<?, ?it/s]

epoch 1/10 : validation


  0%|          | 0/2 [00:00<?, ?it/s]

epoch 2/10 : training


  0%|          | 0/8 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [18]:
import textwrap

In [17]:
# https://huggingface.co/transformers/main_classes/model.html#transformers.generation_utils.GenerationMixin.generate
# модель без дообучения

# prompt = "print('Hello world!') "
prompt = 'У Лукоморья дуб зеленый'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model_init.generate(
    input_ids=prompt,
    max_length=50,
    num_beams=5,
    do_sample=True,
    temperature=10.,
    top_k=50,
    top_p=0.6,
    no_repeat_ngram_size=3,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


У Лукоморья дуб зеленый...  И дуб дубовый, но в поле не поле". - Примеч.: Лукумеж. "Лес" - в "Итоге войны" есть рассказ
"Защитница леса", где
------------------
У Лукоморья дуб зеленый! (И. Гарин. «Наследство из рода Бугаевых») (1936 г.). [7, № 128 (7372).] С детства, как у
каждого мальчика с улицы
------------------
У Лукоморья дуб зеленый  Александр Павлович Бехштейн (16 мая 1789–29 октября 1861) – французский композитор из группы
романсов. Начав исполнять романсы в 1782, композитор продолжал работать как певец в 1802.
------------------
У Лукоморья дуб зеленый? Да как это, а как он туда попал – не известно и все, что можно знать из газетных новостей и
телевизионной информации.  О, да ты уже почти закончила рассказ по книге Лукумара
------------------
У Лукоморья дуб зеленый Морис Женери (Le Gaule de Montreux), Луи Тальберг, Анри Блез Лувуа Жак Монсон-Пирейи Жан Бозон,
Л. Т
------------------
У Лукоморья дуб зеленый (Palacha navila, или Vespicilla)   Сосново–Осенние ягоды (Hypersemon

In [18]:
# дообученная модель

prompt = 'Бродил котенок'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model.generate(
    input_ids=prompt,
    max_length=150,
    num_beams=5,
    do_sample=True,
    temperature=100.,
    top_k=50,
    top_p=0.6,
    no_repeat_ngram_size=2,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 150), end='\n------------------\n')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Бродил котенок без утех — И был он пуст! В нем не зелья в нем дело: Заветную песню ему пели птицы; Спит душа во дворце… Он бродил среди звезд да волн…
Но не в его забавы была сила. Остриё в душе билось! Вдруг раздался глухой звон: Звери за дверью… Схватился кот за нож свой; он поднял с земли лук. Но
нет там ничего! Заблуждение сделал: Как молния пронзет, как стрелы… Как птица летит; О, в сердце у человека не болит — Забудьте, прощай. Теперь — мир
— законный!» «Унесет его сон. Снимут все обиды! Я ухожу. Прощу обиды
------------------
Бродил котенок. И увидел царь зверей, — Что сидит кот ученый: В кресле на стене — череп с кости; он был весь как скелет Из черепа; ноги были покрыты
кожей От шеи вниз на спину и вверх И весь он стал, и хвост его в пыли; Припомнили о нём цари и князья и, собрався в совете Из царства и царицы… Беда
пришла от людей — Храбрости Жестой-Лезвию в голову, В ней погибели — и ярости жестокой Бледнее стал череп этот! Настал страшный голод… Царь голодного
Собачьи мордо

In [19]:
out

array([[  983,   679,   364, ...,  1483,  4724, 26895],
       [  983,   679,   364, ...,  1009,   521,  5799],
       [  983,   679,   364, ...,   546,  2285, 24455],
       ...,
       [  983,   679,   364, ...,   203,  4925,   551],
       [  983,   679,   364, ...,   492,   322,  5072],
       [  983,   679,   364, ...,   289, 41940,    18]])

In [20]:
# def generate(prompt, len_gen=20, temperature=1):
#     generated = tokenizer.encode(prompt)
#     context = torch.tensor([generated]).to(device)
#     past = None

#     for i in tqdm(range(len_gen)):
#         output, past = model(context, past_key_values=past).values()
#         # token = torch.argmax(output[..., -1, :], dim=-1)
#         output = output / temperature
#         token = torch.distributions.Categorical(logits=output[..., -1, :]).sample()
        
#         generated += token.tolist()
#         context = token.unsqueeze(0)

#     sequence = tokenizer.decode(generated)

#     return sequence

In [21]:
torch.save(model.state_dict(), '/content/drive/MyDrive/weights_faitytales')

In [23]:
model_2 = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)
model_2.load_state_dict(torch.load('/content/drive/MyDrive/weights_faitytales'))

model_2.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [24]:
model_2.save_pretrained('/content/drive/MyDrive/model_sber_gpt2_pretrained')
# torch.save(model, 'model.pth')

In [25]:
tokenizer.save_pretrained('/content/drive/MyDrive/model_sber_gpt2_pretrained')

('/content/drive/MyDrive/model_sber_gpt2_pretrained/tokenizer_config.json',
 '/content/drive/MyDrive/model_sber_gpt2_pretrained/special_tokens_map.json',
 '/content/drive/MyDrive/model_sber_gpt2_pretrained/vocab.json',
 '/content/drive/MyDrive/model_sber_gpt2_pretrained/merges.txt',
 '/content/drive/MyDrive/model_sber_gpt2_pretrained/added_tokens.json')

In [16]:
model_3 = GPT2LMHeadModel.from_pretrained(
    '/content/drive/MyDrive/model_sber_gpt2_pretrained',
    output_attentions = False,
    output_hidden_states = False)

model_3.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [97]:
tokenizer = GPT2Tokenizer.from_pretrained('/content/drive/MyDrive/model_sber_gpt2_pretrained')

prompt = 'Катился колобок'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model_3.generate(
    input_ids=prompt,
    max_length=150,
    num_beams=7,
    do_sample=True,
    temperature=200.,
    top_k=50,
    top_p=0.6,
    no_repeat_ngram_size=7,
    num_return_sequences=5,
    ).cpu().numpy()
for out_ in out:
    final_out = tokenizer.decode(out_)
    final_out = final_out.replace('«', '')
    final_out = final_out.replace('»', '')
    final_out = final_out.replace('"', '')
    final_out = final_out.replace('"', '')
    final_out = final_out.replace('(', '')
    final_out = final_out.replace(')', '')
    final_out = re.sub('\d', '', final_out)
    final_out = final_out.replace('[', '')
    final_out = final_out.replace(']', '')
    final_out = final_out.replace('./-..', '')
    final_out = final_out.replace(' .', '.')
    final_out = final_out.replace('/', ' ')
    final_out = final_out.replace('   ', ' ')
    final_out = final_out.replace('  ', ' ')
    final_out = final_out.replace('  ', ' ')
    final_out = final_out.replace('  ', ' ')
    final_out = final_out.replace('  ', ' ')
    final_out = final_out.replace('  ', ' ')
    final_out = final_out.replace('   ', ' ')
    final_out = final_out.replace('                 ', ' ')
    final_out = final_out.replace(';', ',')
    final_out = final_out.replace('?...', '?')
    final_out = final_out.replace('?..', '?')
    final_out = final_out.replace('!...', '!')
    final_out = final_out.replace('!..', '!')
    final_out = final_out.replace('...,', ',')
    final_out = final_out.lower()
    string = ''
    for i, char in enumerate(final_out):
      if i == 0:
        string += char.upper()
      elif (final_out[i-2] == '.' or final_out[i-2] == '!' or final_out[i-2] == '?') and (i >= 2):
        string += char.upper()
      elif final_out[i-4] == '.' and final_out[i-3] == '.' and final_out[i-2] == '.' and i >= 4:
        string += char.upper()
      else:
        string += char
    end_of_sentence = [final_out.rfind("."), final_out.rfind("?"), final_out.rfind("!"), final_out.rfind('"'), final_out.rfind('...')]
    print(textwrap.fill(string[:max(end_of_sentence)+1], 150), end='\n------------------\n')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Катился колобок на брюхо — он с места и пуст, — не спеша к берегу лунтик нес. К реке пристать собрался и увидел там: три коня на санках. Один серый. У
второго черные глаза, один с крючкою носок. И черный тоже черный: он был весь черный, все зубы острые и страшные: весь покрылся черными и черными
буре-пу- дми. На шее два длинных узлы- трех коней белые перья.
------------------
Катился колобок с неба. И вот в тот час к ним приехал старик, как гром, отрока зайду и за руку меня обую! На груди он крест целовал, за уши гладит. А
тот — молью горд и не внимал ничьей своебщностей: то ему мать в руки отдала… и вот идет старик за ним! А конь уж и конь уж! Ну, едет, едет… и видит -
кругом снег. Ну, за ним в след бежит - кругом белый медведь! Куда же ему, куда?
------------------
Катился колобок, в окошко, и из него в окно падал, как снег из кулуков воспел и за окном катилась… и вот постучалась дверь! Вскочил старик на постели
с палкой да на двор свой пустил… вмиг сбежали из избы бесчисленными воро